<a href="https://colab.research.google.com/github/kgene521/Calc/blob/master/ekSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# !wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv
!mkdir spark321
!tar xf spark-3.2.1-bin-hadoop3.2.tgz -C spark321/
!wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv
!mv cars.csv spark321/
!ls -la spark321/
!cd spark321/;pwd
!cd /content/
!pip install -q findspark
!cd /content/spark321/;pwd;ls -la
!pip install pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark321/spark-3.2.1-bin-hadoop3.2"
!echo $SPARK_HOME

In [ ]:
import os
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", 'True') # Property used to format output tables better
# spark
df = spark.read.csv('/content/spark321/cars.csv', header=True, sep=";")
# df.show(5, truncate=False)
df2 = df.limit(5)
# df2
# df2.columns
# df2.dtypes
df2.printSchema
df.limit(5)
# Creating a list of the schema in the format column_name, data_type
from pyspark.sql.types import *
labels = [
     ('Car',StringType()),
     ('MPG',DoubleType()),
     ('Cylinders',IntegerType()),
     ('Displacement',DoubleType()),
     ('Horsepower',DoubleType()),
     ('Weight',DoubleType()),
     ('Acceleration',DoubleType()),
     ('Model',IntegerType()),
     ('Origin',StringType())
]
# # Creating the schema that will be passed when reading the csv
# print(labels)
schema = StructType([StructField (x[0], x[1], True) for x in labels])
# schema
df = spark.read.csv('/content/spark321/cars.csv', header=True, sep=";", schema=schema)
# df.printSchema()
# df.show(truncate=False)
# print(df.Car)
# print("*"*35)
# df.select(df.Car).show(truncate=False)
# print(df['car'])
# print("*"*20)
# Column name is case sensitive
# df.select(df['car']).show(truncate=False)

from pyspark.sql.functions import col
### Column name is case insensitive
# df.select(col('car'), col('CAR')).show(truncate=False)
### 1st method
### Column name is case sensitive in this usage
### print(df.Car, df.Cylinders)
### print("*"*40)
# df.select(df.Car, df.Cylinders).show(truncate=False)
### print(df2)
### 2nd method
### Column name is case insensitive in this usage
### print(df['car'],df['cylinders'])
### print("*"*40)
### df.select(df['caR'],df['cylINDers']).show(truncate=False)
### 3rd method
### Column name is case insensitive in this usage
from pyspark.sql.functions import col
# df.select(col('caR'),col('cylINDers')).show(truncate=False)
### CASE 1: Adding a new column
### We will add a new column called 'first_column' at the end
from pyspark.sql.functions import lit
df = df.withColumn('first_column',lit(1))
### lit means literal. It populates the row with the literal value given.
### When adding static data / constant values, it is a good practice to use it.
# df.show(5,truncate=False)
# CASE 2: Adding multiple columns
# We will add two new columns called 'second_column' and 'third_column' at the end
df = df.withColumn('second_column', lit(2)) \
       .withColumn('third_column', lit('Third Column'))
# lit means literal. It populates the row with the literal value given.
# When adding static data / constant values, it is a good practice to use it.
# df.show(5,truncate=False)
# CASE 3: Deriving a new column from an exisitng one
# We will add a new column called 'car_model' which has the value of car and model appended together with a space in between 
from pyspark.sql.functions import concat
df = df.withColumn('car_model', concat(col("Car"), lit(" "), col("model")))
# lit means literal. It populates the row with the literal value given.
# When adding static data / constant values, it is a good practice to use it.
# df.show(5,truncate=False)
#Renaming a column in PySpark
df = df.withColumnRenamed('first_column', 'new_column_one') \
       .withColumnRenamed('second_column', 'new_column_two') \
       .withColumnRenamed('third_column', 'new_column_three')
# df.show(truncate=False)

# Group By a column in PySpark
# df.groupBy('Origin').count().show(5)

# Group By multiple columns in PySpark
df.groupBy('Origin', 'Model').count().show(5)
#Remove columns in PySpark
df = df.drop('new_column_one')
# df.show(5,truncate=False)

#Remove multiple columnss in one go
df = df.drop('new_column_two') \
       .drop('new_column_three')
# df.show(5,truncate=False)


# Filtering rows in PySpark
total_count = df.count()
print("TOTAL RECORD COUNT: " + str(total_count))
europe_filtered_count = df.filter(col('Origin')=='Europe').count()
print("EUROPE FILTERED RECORD COUNT: " + str(europe_filtered_count))
# df.filter(col('Origin')=='Europe').show(truncate=False)
# Filtering rows in PySpark based on Multiple conditions
total_count = df.count()
print("TOTAL RECORD COUNT: " + str(total_count))
europe_filtered_count = df.filter((col('Origin')=='Europe') &
                                  (col('Cylinders')==4)).count() # Two conditions added here
print("EUROPE FILTERED RECORD COUNT: " + str(europe_filtered_count))
# df.filter(col('Origin')=='Europe').show(truncate=False)

#Get Unique Rows in PySpark
# df.select('Origin').distinct().show()



#Get Unique Rows in PySpark based on mutliple columns
# df.select('Origin','model').distinct().show()



# Sort Rows in PySpark
# By default the data will be sorted in ascending order
# df.orderBy('Cylinders').show(5, truncate=False)
# df.orderBy('Horsepower', ascending=False).show(5, truncate=False)

# Using groupBy aand orderBy together
df.groupBy("Origin").count().orderBy('count', ascending=False).show(10)

# CASE 1: Union When columns are in order
europe_cars = df.filter((col('Origin')=='Europe') & (col('Cylinders')==5))
japan_cars = df.filter((col('Origin')=='Japan') & (col('Cylinders')==3))
print("EUROPE CARS: "+str(europe_cars.count()))
print("JAPAN CARS: "+str(japan_cars.count()))
print("AFTER UNION: "+str(europe_cars.union(japan_cars).count()))

# CASE 1: Union When columns are not in order
# Creating two dataframes with jumbled columns
df1 = spark.createDataFrame([[1, 2, 3]], ["col0", "col1", "col2"])
df2 = spark.createDataFrame([[4, 5, 6]], ["col1", "col2", "col0"])
df1.unionByName(df2).show()

# Functions available in PySpark
from pyspark.sql import functions
# Similar to python, we can use the dir function to view the avaiable functions
foos = dir(functions)
counter = 0
for x in foos:
  counter = counter + 1
  if x[0] == '_' or counter <= 8:
    continue
  print(f"x: {x}")



In [126]:
foos = dir(functions)
function_list = ""
counter = 0
for x in foos:
  counter = counter + 1
  if x[0] == '_' or counter <= 8:
    continue
  if counter % 10 == 0:
    print(function_list)
    function_list = ""
  else:
    function_list = function_list + ", " + x
print(function_list)
# print(foos)






, abs, acos, acosh, add_months, aggregate, approxCountDistinct, approx_count_distinct, array, array_contains
, array_except, array_intersect, array_join, array_max, array_min, array_position, array_remove, array_repeat, array_sort
, arrays_overlap, arrays_zip, asc, asc_nulls_first, asc_nulls_last, ascii, asin, asinh, assert_true
, atan2, atanh, avg, base64, bin, bitwiseNOT, bitwise_not, broadcast, bround
, cbrt, ceil, coalesce, col, collect_list, collect_set, column, concat, concat_ws
, corr, cos, cosh, count, countDistinct, count_distinct, covar_pop, covar_samp, crc32
, cume_dist, current_date, current_timestamp, date_add, date_format, date_sub, date_trunc, datediff, dayofmonth
, dayofyear, days, decode, degrees, dense_rank, desc, desc_nulls_first, desc_nulls_last, element_at
, exists, exp, explode, explode_outer, expm1, expr, factorial, filter, first
, floor, forall, format_number, format_string, from_csv, from_json, from_unixtime, from_utc_timestamp, functools
, greatest, grouping, 